# Створення нових українських слів


### Реалізуємо RNN на рівні символів

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import random

ukrainian_words = [
    "день", "ніч", "сонце", "мило", "країна", "любов", "дружба",
    "справа", "визивати", "робота", "комп'ютер", "вікно", "книга",
    "автомобіль", "філм", "музика", "природа", "заклад", "ідея",
    "програмування", "інновація", "екосистема", "інтелект", "комунікація",
    "інформація", "економіка", "політика", "культурний", "історичний",
    "науковий", "технологія", "інженер", "архітектор", "дизайнер",
    "програміст", "веб-розробник", "дата", "аналітик", "менеджер",
    "маркетинг", "реклама", "продаж", "кінцевий", "результат", "досягнення",
    "стратегія", "планування", "організація", "координація", "командування",
    "комунікація", "зворотній", "підсумок", "дискусія", "колаборативний",
    "інноваційний", "діяльність", "функціонал", "інтеграція", "адаптація",
    "скалкування", "перспективний", "потенціал", "реалізація", "імплементація",
    "оптимізація", "автоматизація", "децентрализований", "блокчейн", "криптовалюта",
    "інтернет", "перевага", "комплексний", "мультіфункціональний", "інноваційний",
    "когнітивний", "інтелектуальний", "компетентний", "професійний", "експеримент",
    "інновації", "дослідження", "аналіз", "синергетичний", "інтеграційний",
    "комунікаційний", "інформаційний", "комп'ютерний", "програмний", "системний",
    "база", "даних", "навчання", "машинний", "підприємство", "бізнес",
    "інвестиції", "фінансовий", "корпоративний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний",
    "маркетинговий", "рекламний", "продажний", "маркетинговий", "рекламний",
    "продажний", "маркетинговий", "рекламний", "продажний", "маркетинговий",
    "рекламний", "продажний", "маркетинговий", "рекламний", "продажний"
]
chars = sorted(set(''.join(ukrainian_words)))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for idx, char in enumerate(chars)}
vocab_size = len(chars)

seq_length = 5
inputs = []
targets = []
for word in ukrainian_words:
    word = word.strip()
    for i in range(len(word) - seq_length):
        inputs.append([char_to_idx[c] for c in word[i:i+seq_length]])
        targets.append(char_to_idx[word[i+seq_length]])

input_tensor = torch.tensor(inputs, dtype=torch.long)
target_tensor = torch.tensor(targets, dtype=torch.long)

class WordDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

dataset = WordDataset(input_tensor, target_tensor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

class CharRNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.lstm(embedded, hidden)
        output = self.fc(output[:, -1, :])
        return output, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))

embedding_size = 100
hidden_size = 128
num_epochs = 100
learning_rate = 0.001

model = CharRNN(vocab_size, embedding_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0
    for inputs_batch, targets_batch in dataloader:
        hidden = model.init_hidden(inputs_batch.size(0))

        hidden = (hidden[0].detach(), hidden[1].detach())

        output, hidden = model(inputs_batch, hidden)
        loss = criterion(output, targets_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

def generate_word(model, seed, max_length=10):
    model.eval()
    with torch.no_grad():
        seed_input = [char_to_idx[c] for c in seed]
        input_seq = torch.tensor(seed_input, dtype=torch.long).unsqueeze(0)
        hidden = model.init_hidden(1)
        generated = seed_input.copy()

        for _ in range(max_length - len(seed)):
            output, hidden = model(input_seq, hidden)
            prob = nn.functional.softmax(output[0], dim=0)
            top_prob, top_idx = torch.topk(prob, k=3)
            selected_idx = random.choices(top_idx.tolist(), weights=top_prob.tolist(), k=1)[0]
            generated.append(selected_idx)
            input_seq = torch.tensor([selected_idx], dtype=torch.long).unsqueeze(0)

        generated_word = ''.join([idx_to_char[idx] for idx in generated])
        return generated_word

seed = "ге"
new_word = generate_word(model, seed, max_length=15)
print(f"Generated word: {new_word}")

Epoch [1/100], Loss: 1.7122
Epoch [2/100], Loss: 0.7629
Epoch [3/100], Loss: 0.5855
Epoch [4/100], Loss: 0.4729
Epoch [5/100], Loss: 0.3890
Epoch [6/100], Loss: 0.3214
Epoch [7/100], Loss: 0.2612
Epoch [8/100], Loss: 0.2097
Epoch [9/100], Loss: 0.1687
Epoch [10/100], Loss: 0.1314
Epoch [11/100], Loss: 0.1041
Epoch [12/100], Loss: 0.0835
Epoch [13/100], Loss: 0.0699
Epoch [14/100], Loss: 0.0573
Epoch [15/100], Loss: 0.0502
Epoch [16/100], Loss: 0.0445
Epoch [17/100], Loss: 0.0410
Epoch [18/100], Loss: 0.0373
Epoch [19/100], Loss: 0.0334
Epoch [20/100], Loss: 0.0326
Epoch [21/100], Loss: 0.0309
Epoch [22/100], Loss: 0.0289
Epoch [23/100], Loss: 0.0290
Epoch [24/100], Loss: 0.0268
Epoch [25/100], Loss: 0.0265
Epoch [26/100], Loss: 0.0257
Epoch [27/100], Loss: 0.0247
Epoch [28/100], Loss: 0.0239
Epoch [29/100], Loss: 0.0232
Epoch [30/100], Loss: 0.0239
Epoch [31/100], Loss: 0.0231
Epoch [32/100], Loss: 0.0223
Epoch [33/100], Loss: 0.0219
Epoch [34/100], Loss: 0.0221
Epoch [35/100], Loss: 0

### UD Ukrainian corpus

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import re

!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Ukrainian-IU/master/uk_iu-ud-train.conllu -O uk_iu-ud-train.conllu


--2024-11-28 08:13:41--  https://raw.githubusercontent.com/UniversalDependencies/UD_Ukrainian-IU/master/uk_iu-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13165569 (13M) [text/plain]
Saving to: ‘uk_iu-ud-train.conllu’

uk_iu-ud-train.conl 100%[===================>]  12.56M  32.0MB/s    in 0.4s    

2024-11-28 08:13:42 (32.0 MB/s) - ‘uk_iu-ud-train.conllu’ saved [13165569/13165569]



### Завантажуємо та виділяємо слова та їх морфологічні ознаки

In [3]:
words = []
word_features = []

with open('uk_iu-ud-train.conllu', 'r', encoding='utf-8') as f:
    for line in f:
        if line.startswith('#') or line.strip() == '':
            continue
        parts = line.strip().split('\t')
        if len(parts) != 10:
            continue
        if '-' in parts[0] or '.' in parts[0]:
            continue
        word = parts[1]
        feats = parts[5]
        pos = parts[3]

        if word.isalpha():
            word = word.lower()
            gender = None
            match = re.search(r'Gender=([A-Za-z]+)', feats)
            if match:
                gender = match.group(1)
            words.append(word)
            word_features.append({'pos': pos, 'gender': gender})


### Прибираємо дублікати

In [4]:
unique_words = set()
unique_data = []
for word, feat in zip(words, word_features):
    if word not in unique_words and len(word) > 2:
        unique_words.add(word)
        unique_data.append((word, feat))

words = [wd[0] for wd in unique_data]
word_features = [wd[1] for wd in unique_data]


### Відображення символів

In [5]:
chars = sorted(list(set(''.join(words))))
chars.extend(['<S>', '<E>', '<UNK>'])
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
vocab_size = len(chars)

### теги POS і стать

In [6]:
pos_tags = sorted(list(set([feat['pos'] for feat in word_features])))
genders = sorted(list(set([feat['gender'] for feat in word_features if feat['gender'] is not None])))

### Створення відображень

In [7]:
pos_to_idx = {pos: idx for idx, pos in enumerate(pos_tags)}
gender_to_idx = {gender: idx for idx, gender in enumerate(genders)}


Створення датасету

In [8]:
def build_dataset(words, word_features, block_size):
    X_char, X_pos, X_gender, Y = [], [], [], []
    for word, feat in zip(words, word_features):
        context = [stoi['<S>']] * block_size
        pos_idx = pos_to_idx[feat['pos']]
        gender_idx = gender_to_idx.get(feat['gender'], -1)
        for ch in word + '<E>':
            ix = stoi.get(ch, stoi['<UNK>'])
            X_char.append(context)
            X_pos.append(pos_idx)
            X_gender.append(gender_idx)
            Y.append(ix)
            context = context[1:] + [ix]
    X_char = torch.tensor(X_char)
    X_pos = torch.tensor(X_pos)
    X_gender = torch.tensor(X_gender)
    Y = torch.tensor(Y)
    return X_char, X_pos, X_gender, Y

block_size = 5
X_char, X_pos, X_gender, Y = build_dataset(words, word_features, block_size)


### Визначення моделі

In [9]:
class WordLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, pos_size, gender_size, feature_dim=32):
        super(WordLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_embedding = nn.Embedding(pos_size, feature_dim)
        self.gender_embedding = nn.Embedding(gender_size + 1, feature_dim)
        self.lstm = nn.LSTM(embedding_dim + 2 * feature_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x_char, x_pos, x_gender, hidden=None):
        embeds = self.embedding(x_char)
        pos_embeds = self.pos_embedding(x_pos).unsqueeze(1).repeat(1, x_char.size(1), 1)
        gender_embeds = self.gender_embedding(x_gender + 1).unsqueeze(1).repeat(1, x_char.size(1), 1)
        lstm_input = torch.cat([embeds, pos_embeds, gender_embeds], dim=2)
        out, hidden = self.lstm(lstm_input, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

embedding_dim = 50
hidden_dim = 128
feature_dim = 32
pos_size = len(pos_tags)
gender_size = len(genders)
model = WordLSTM(vocab_size, embedding_dim, hidden_dim, pos_size, gender_size, feature_dim)


### Тренуємо модель

In [10]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch_size = 64
epochs = 20

for epoch in range(epochs):
    epoch_loss = 0
    permutation = torch.randperm(X_char.size()[0])

    for i in range(0, X_char.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_x_char = X_char[indices]
        batch_x_pos = X_pos[indices]
        batch_x_gender = X_gender[indices]
        batch_y = Y[indices]

        model.zero_grad()
        output, _ = model(batch_x_char, batch_x_pos, batch_x_gender)
        loss = loss_function(output, batch_y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * batch_x_char.size(0)

    average_loss = epoch_loss / len(X_char)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {average_loss:.4f}')


Epoch 1/20, Loss: 1.8155
Epoch 2/20, Loss: 1.6117
Epoch 3/20, Loss: 1.5398
Epoch 4/20, Loss: 1.4895
Epoch 5/20, Loss: 1.4507
Epoch 6/20, Loss: 1.4179
Epoch 7/20, Loss: 1.3908
Epoch 8/20, Loss: 1.3665
Epoch 9/20, Loss: 1.3462
Epoch 10/20, Loss: 1.3280
Epoch 11/20, Loss: 1.3115
Epoch 12/20, Loss: 1.2966
Epoch 13/20, Loss: 1.2823
Epoch 14/20, Loss: 1.2702
Epoch 15/20, Loss: 1.2580
Epoch 16/20, Loss: 1.2479
Epoch 17/20, Loss: 1.2381
Epoch 18/20, Loss: 1.2296
Epoch 19/20, Loss: 1.2206
Epoch 20/20, Loss: 1.2125


### Утворення слів за допомогою фільтрів

In [11]:
def generate_word(model, stoi, itos, block_size, pos_filter=None, gender_filter=None, max_length=12):
    model.eval()
    with torch.no_grad():
        context = torch.tensor([[stoi['<S>']] * block_size])
        word = ''
        hidden = None

        if pos_filter is not None and pos_filter in pos_to_idx:
            pos_idx = torch.tensor([pos_to_idx[pos_filter]])
        else:
            pos_idx = torch.tensor([np.random.choice(len(pos_tags))])
        if gender_filter is not None and gender_filter in gender_to_idx:
            gender_idx = torch.tensor([gender_to_idx[gender_filter]])
        else:
            gender_idx = torch.tensor([np.random.choice(len(genders)) - 1])

        for _ in range(max_length):
            output, hidden = model(context, pos_idx, gender_idx)
            prob = torch.softmax(output, dim=1)
            char_ind = torch.multinomial(prob, num_samples=1).item()
            char = itos[char_ind]
            if char == '<E>':
                break
            word += char
            context = torch.cat([context[:, 1:], torch.tensor([[char_ind]])], dim=1)
        return word


In [12]:
print("Available POS tags:", pos_tags)
print("Available genders:", genders)

#  feminine nouns
for _ in range(5):
    word = generate_word(model, stoi, itos, block_size, pos_filter='NOUN', gender_filter='Fem')
    print(f"Generated feminine noun: {word}")

#  verbs
for _ in range(5):
    word = generate_word(model, stoi, itos, block_size, pos_filter='VERB')
    print(f"Generated verb: {word}")


Available POS tags: ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'SCONJ', 'VERB', 'X']
Available genders: ['Fem', 'Masc', 'Neut']
Generated feminine noun: філкувами<UNK><UNK><UNK>
Generated feminine noun: загреснях<UNK><UNK><UNK>
Generated feminine noun: молина<UNK><UNK><UNK><UNK><UNK><UNK>
Generated feminine noun: мати<UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK>
Generated feminine noun: співпрексуті
Generated verb: записаєш<UNK><UNK><UNK><UNK>
Generated verb: зоблюкувати<UNK>
Generated verb: моржлала<UNK><UNK><UNK><UNK>
Generated verb: підобрали<UNK><UNK><UNK>
Generated verb: поїждять<UNK><UNK><UNK><UNK>
